In [1]:
import pytorch_lightning as pl
import pandas as pd
import os 
import torch
import torchvision
from torch.utils.data import Dataset ,DataLoader
from torchmetrics import Accuracy
from pytorch_lightning.callbacks import ModelCheckpoint
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models
# from lightning.pytorch.loggers import WandbLogger
from pytorch_lightning.loggers import WandbLogger
import wandb

In [2]:




# use directly wandb module
# wandb.config["key"] = "a2e6402ce9fe2ebe1f01d5332c4fafa210b0dc0c"
# wandb.config.update()



wandb.login(key="a2e6402ce9fe2ebe1f01d5332c4fafa210b0dc0c")
pName = "Assignment 2 Part A main 4"
run_obj=wandb.init( project=pName)
#     obj = CNNModel(config.activation_function,config.batch_normalization,config.data_augmentation,config.filter_organisation, config.drop_out)

# Set up the configuration for your sweep using the `wandb.sweep` function
sweep_config = {
    'method': 'bayes', # or 'grid'
    'metric': {'name': 'val_loss', 'goal': 'minimize'},
    'parameters': {
        'drop_out': {"values": [0.2, 0.3]},
        "activation_function": {
              "values": [ "ReLU", "SiLU", "GELU", "Mish"]
          },
          
          "learning_rate": {
              "values": [1e-3, 1e-4]
          }
          ,
        "filter_organisation":{
            "values":[[8,8,8,8,8],[16,16,16,16,16],[32,32,32,32,32],[64,64,64,64,64]]
        },
        "data_augmentation":{
            "values":["Yes","No"]
        },
        "batch_normalization":{
            "values":["Yes","No"]
        },
          "epochs": {
              "values": [5, 10]
          },
    }
}



wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs22m008. Use `wandb login --relogin` to force relogin


CNN model 

In [3]:
class CNNModel(pl.LightningModule):
    
    def __init__(self,activation_function,batch_normalization,data_augmentation,filter_organisation,drop_out):
        self.activation_function=activation_function
        self.batch_normalization=batch_normalization
#         self.data_augmentation=data_augmentation
        super(CNNModel, self).__init__()
        
        self.cnv1 = torch.nn.Conv2d(3, filter_organisation[0], 3)
        self.cnv2 = torch.nn.Conv2d(filter_organisation[0], filter_organisation[1], 3)
        self.cnv3 = torch.nn.Conv2d(filter_organisation[1], filter_organisation[2], 3)      
        self.cnv4 = torch.nn.Conv2d(filter_organisation[2], filter_organisation[3], 3)
        self.cnv5 = torch.nn.Conv2d(filter_organisation[3], filter_organisation[4], 3)
        
        if(activation_function=="ReLU"):
            self.activation_function=nn.ReLU()
        elif(activation_function=="GELU"):
            self.activation_function=nn.GELU()
        elif(activation_function=="SiLU"):
            self.activation_function=nn.SiLU()
        elif(activation_function=="Mish"):
            self.activation_function=nn.Mish()
        
        stride=2
        input_size=256
        
        DenseLayerSize=input_size
        for filter in filter_organisation:
            DenseLayerSize = (DenseLayerSize-4)//stride + 1
        
        self.bn = nn.BatchNorm1d(DenseLayerSize*DenseLayerSize*filter_organisation[4])
        self.mxpool = nn.MaxPool2d(2)
        self.flat = nn.Flatten()
        self.fc= nn.Linear(DenseLayerSize*DenseLayerSize*filter_organisation[4],10)
        self.softmax = nn.Softmax()
        self.learning_rate=0.001
        self.dropout = nn.Dropout(p=drop_out)
        self.save_hyperparameters()
#         self.batch_normalization=batch_normalization

    def forward(self,x):
#         print(x.shape)
        out=self.activation_function(self.cnv1(x))
        out=self.mxpool(out)
#         print("print: mp1    ",out.shape)

        out=self.activation_function(self.cnv2(out))
        out=self.mxpool(out)
#         print("print: mp2    ",out.shape)

        out=self.activation_function(self.cnv3(out))
        out=self.mxpool(out)
#         print("print: mp3    ",out.shape)

        out=self.activation_function(self.cnv4(out))
        out=self.mxpool(out)
#         print("print: mp4    ",out.shape)

        out=self.activation_function(self.cnv5(out))
        
        out = self.mxpool(out)
#         print("print: mp5    ",out.shape)
        out=self.flat(out)
#         print("print: result    ",out.shape)
        if(self.batch_normalization=="Yes"):
            out=self.bn(out) 
        out = self.dropout(out)

        out = self.activation_function(self.fc(out))
        return out
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
        self.log("train_accuracy", accuracy, prog_bar=True,on_epoch=True)
#         wandb.log({"train/loss": loss})
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        self.log("val_loss", loss,prog_bar=True, on_epoch=True)
#         print("val Loss:",loss)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
#         print("val_accuracy: ",accuracy)
        self.log("val_accuracy", accuracy, prog_bar=True,on_epoch=True)

    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
#         print("test_accuracy: ",accuracy)
        self.log("test_loss", loss, prog_bar=True, on_epoch=True)
        self.log("test_accuracy", accuracy, prog_bar=True,on_epoch=True)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer
    

In [4]:


# # # # testing
# # Define data transformations
# transform = transforms.Compose([
#     transforms.Resize((256,256)),

#     transforms.ToTensor(),

#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                         std=[0.229, 0.224, 0.225])
# ])

# transform_augmented = transforms.Compose([
#     transforms.Resize((256,256)),
#     transforms.AutoAugment(),

#     transforms.ToTensor(),

#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                         std=[0.229, 0.224, 0.225])
# ])



# dataset = datasets.ImageFolder('/kaggle/input/inaturalist12k/Data/inaturalist_12K/train', transform=transform_augmented)

# test_dataset = datasets.ImageFolder('/kaggle/input/inaturalist12k/Data/inaturalist_12K/val', transform=transform)



# # Split dataset into training and testing sets
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset)-int(0.8*len(dataset))])

# # Create data loader objects for training and testing sets
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
# val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)



# obj = CNNModel("ReLU","Yes","Yes",[64,64,64,64,32], 0.3)
# trainer = pl.Trainer(max_epochs=3, accelerator="gpu", devices=1)
# trainer.fit(model=obj,train_dataloaders=train_dataloader,val_dataloaders=val_dataloader)

In [5]:
def sweep():
    wandb.init()
# sweep_id = wandb.sweep(sweep_config,project="Asg 1 Final")
# wandb.agent(sweep_id, sweep)
    wandb_logger = WandbLogger()
    config = wandb.config

    # Define data transformations
    transform = transforms.Compose([
        transforms.Resize((256,256)),

        transforms.ToTensor(),

        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])

    transform_augmented = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.AutoAugment(),

        transforms.ToTensor(),

        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
    ])


    # Load dataset from directory
    if(config.data_augmentation=="No"):
        dataset = datasets.ImageFolder('/kaggle/input/inaturalist12k/Data/inaturalist_12K/train', transform=transform)
    else:
        dataset = datasets.ImageFolder('/kaggle/input/inaturalist12k/Data/inaturalist_12K/train', transform=transform_augmented)

    test_dataset = datasets.ImageFolder('/kaggle/input/inaturalist12k/Data/inaturalist_12K/val', transform=transform)



    # Split dataset into training and testing sets
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset)-int(0.8*len(dataset))])

    # Create data loader objects for training and testing sets
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)


    print(config.activation_function,config.batch_normalization,config.data_augmentation,config.filter_organisation, config.drop_out)
    
    obj = CNNModel(config.activation_function,config.batch_normalization,config.data_augmentation,config.filter_organisation, config.drop_out)
    
    trainer = pl.Trainer(max_epochs=config.epochs, accelerator="gpu", devices=1, logger=wandb_logger)
    
    trainer.fit(model=obj,train_dataloaders=train_dataloader,val_dataloaders=val_dataloader)
    # trainer.test(model=net, datamodule=dataset, ckpt_path='best')

    run_name = "lr_{}_ac_{}_bn_{}_da_{}_bs_{}_dp_{}".format(config.learning_rate, config.activation_function,config.batch_normalization,config.data_augmentation,config.filter_organisation, config.drop_out)
    print(run_name)
    wandb.run.name = run_name
    wandb.run.save()
#     wandb.finish()

# Initialize the WandB sweep
sweep_id = wandb.sweep(sweep_config,project=pName)
wandb.agent(sweep_id, sweep)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: aysozaak
Sweep URL: https://wandb.ai/cs22m008/Assignment%202%20Part%20A%20main%202.9/sweeps/aysozaak


wandb: Waiting for W&B process to finish... (success).
wandb: 🚀 View run stellar-sea-1 at: https://wandb.ai/cs22m008/Assignment%202%20Part%20A%20main%202.9/runs/3fvq45v8
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230410_194954-3fvq45v8/logs
wandb: Agent Starting Run: a31ybp0u with config:
wandb: 	activation_function: SiLU
wandb: 	batch_normalization: Yes
wandb: 	data_augmentation: Yes
wandb: 	drop_out: 0.2
wandb: 	epochs: 10
wandb: 	filter_organisation: [16, 16, 16, 16, 16]
wandb: 	learning_rate: 0.001


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"


SiLU Yes Yes [16, 16, 16, 16, 16] 0.2


...

Training: 0it [00:00, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.


Error in callback <function _WandbInit._pause_backend at 0x7a9986018710> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

epoch,▁▁
train_accuracy_step,▁█
train_loss_step,█▁
trainer/global_step,▁█
epoch,0
train_accuracy_step,0.28125
train_loss_step,2.05953
trainer/global_step,99
